In [1]:
training_data = [
    ['Green',3,'Apple'],
    ['Yellow',3,'Apple'],
    ['Red',1,'Grape'],
    ['Red',1,'Grape'],
    ['Yellow',3,'Lemon'],
]

In [4]:
def unique_values(data,col):
    return set([row[col] for row in data])

unique_values(training_data,0)

{'Green', 'Red', 'Yellow'}

In [5]:
def class_counts(data):
    counts={}
    for row in data:
        label = row[-1]
        if label not in counts:
            counts[label]=0
        counts[label]+=1
    return counts

class_counts(training_data)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

In [6]:
header = ["Color","Diameter","label"]

In [7]:
class Question:
    def __init__(self,column,value): #1,3 => Is diameter == 3?
        self.column=column
        self.value=value
    def match(self,example): #Green 3 Apple
        val = example[self.column]#3
        return val == self.value #3 == 3
    def __repr__(self):
        return "Is %s %s %s?"%(header[self.column],"==",str(self.value))

In [8]:
Question(0,'Red')

Is Color == Red?

In [9]:
Question(1,3)

Is Diameter == 3?

In [17]:
q = Question(1,3)
q.match(training_data[2])

False

In [18]:
def partition(rows,question):
    true_rows,false_rows = [],[]
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows,false_rows

In [19]:
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl]/float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

print(gini(training_data))

0.6399999999999999


In [20]:
true_rows,false_rows = partition(training_data, Question(1,3))

In [21]:
true_rows

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

In [22]:
false_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [23]:
def info_gain(left,right,uncertainity):
    p = float(len(left)) / (len(left) + len(right))
    q = float(len(right)) / (len(left) + len(right))
    return uncertainity - p*gini(left) - q*gini(right)

In [24]:
info_gain(true_rows,false_rows,gini(training_data))

0.37333333333333324

In [25]:
true_rows,false_rows = partition(training_data, Question(0,"Red"))

In [26]:
info_gain(true_rows,false_rows,gini(training_data))

0.37333333333333324

In [27]:
def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_uncertainity = gini(rows)
    n_features = len(rows[0])-1
    for col in range(n_features):
        values = unique_values(rows,col)
        for val in values:
            question = Question(col,val)
            true_rows,false_rows = partition(rows,question)
            
            if len(true_rows)==0 or len(false_rows)==0:
                continue
            
            gain = info_gain(true_rows,false_rows,current_uncertainity)
            
            if gain >= best_gain:
                best_gain,best_question = gain,question
    return best_gain,best_question

In [28]:
best_gain,best_question = find_best_split(training_data)
print(best_gain)
print(best_question)

0.37333333333333324
Is Diameter == 3?


In [47]:
class Leaf:
    def __init__(self,rows):
        self.predictions = class_counts(rows)

In [48]:
class Decision_Node:
    def __init__(self,question,true_branch,false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [49]:
def build_tree(rows):
    gain,question = find_best_split(rows)
    if gain==0:
        return Leaf(rows)
    true_rows,false_rows = partition(rows,question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return Decision_Node(question,true_branch,false_branch)

In [50]:
mytree = build_tree(training_data)
print(mytree.true_branch.true_branch.predictions)

{'Apple': 1, 'Lemon': 1}


In [51]:
def print_tree(node,spacing=" "):
    if isinstance(node, Leaf):
        print(spacing+"Predict",node.predictions)
        return
    print(spacing + str(node.question))
    
    print(spacing + '-->True:')
    print_tree(node.true_branch,spacing+"    ")
    
    print(spacing + '-->False:')
    print_tree(node.false_branch,spacing+"    ")

In [57]:
print_tree(mytree)

 Is Diameter == 3?
 -->True:
     Is Color == Yellow?
     -->True:
         Predict {'Apple': 1, 'Lemon': 1}
     -->False:
         Predict {'Apple': 1}
 -->False:
     Predict {'Grape': 2}


In [58]:
def print_leaf(counts):
    total = sum(counts.values())*1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl]/total * 100)) + "%"
    return probs

In [59]:
def classify(row,node):
    if isinstance(node,Leaf):
        return node.predictions
    
    if node.question.match(row):
        return classify(row,node.true_branch)
    else:
        return classify(row,node.false_branch)

In [60]:
testing_data = [
    ['Red',1,'Apple'],
    ['Yellow',3,'Apple'],
]

In [61]:
def test(testing_data):
    global mytree
    for row in testing_data:
        print("Actual: %s. Predicted: %s."%(row[-1],print_leaf(classify(row,mytree))))
    for row in testing_data:
        training_data.append(row)
    mytree = build_tree(training_data)
    print_tree(mytree)

In [64]:
test(testing_data)

Actual: Cherry. Predicted: {'Grape': '66%', 'Apple': '33%'}.
Actual: Mango. Predicted: {'Apple': '66%', 'Lemon': '33%'}.
 Is Diameter == 3?
 -->True:
     Is Color == Yellow?
     -->True:
         Predict {'Apple': 2, 'Lemon': 1, 'Mango': 1}
     -->False:
         Predict {'Apple': 1}
 -->False:
     Predict {'Grape': 2, 'Apple': 1, 'Cherry': 1}


In [63]:
testing_data = [
    ['Red',1,'Cherry'],
    ['Yellow',3,'Mango'],
]